# Featurizer

In [5]:
# general stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Imports
import traceback
import numpy as np
import warnings
import matplotlib.pyplot as plt    
import ipywidgets as widgets
import matplotlib.image as mpimg
import numpy.fft as fft
import threading
import time
import types
import scipy.signal
import signal
import sh
import sys
import os
import re
import struct
import visa
import logging
import copy
import glob
import pandas as pd
import pickle
from IPython.display import display

sys.path.append('/home/colindv/sciebo/masterthesis/remote control/Remote control v3 (version control)/')
home = os.environ['HOME']+'/' # home dir

from colors import color_array
from helper_functions import read_binary
from helper_functions import info, debug, warning, error, critical
from ContextManager import ContextManager
from Receiver import Receiver

qnap = "/storage/qnap/"
folder = "interference_free/cable/no_cfo/"
npy_folder = "npy/"

mod_schemes = ["SC_BPSK", "SC_QPSK", "SC_16QAM", "SC_64QAM",
               "OFDM_BPSK", "OFDM_QPSK", "OFDM_16QAM", "OFDM_64QAM"]

dummyReceiver = Receiver(**{'bin_dir': '/home/colindv/bin_data/', 
                            'freq': 5750000000.0, 
                            'srate': 50000000.0, 
                            'name': 'dummy', 
                            'rx_dir': '/home/colindv/bin_data/', 
                            'bw_noise': 5000000.0, 
                            'bw': 100000.0, 
                            'vbw': 10000.0, 
                            'init_gain': 0, 
                            'freq_noise': np.array([5.73e9, 5.77e9]),
                            'span': 50000000.0, 
                            'max_gain': 0, 
                            'freq_noise_offset': 20000000.0, 
                            'bw_signal': 20000000.0})

dummyReceiver.name = 'dummy'

# Parameters
T_s = 1/dummyReceiver.srate # sampling time
A_t = 1 # threshold
N_s = 1000 # window length FFT

def drop_redundant_cols(df):

    nunique = df.apply(pd.Series.nunique)
    redundant_labels = nunique[nunique == 1].index
    return df.drop(redundant_labels, axis=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def featurize_row(df, r, plot=False, N_t=1000):
    
    N = r.size
    x = np.arange(N)
    r_star = np.conj(r)
    # zero-mean
    r10 = r - np.mean(r)
    r11 = r_star - np.mean(r_star)
    
    # zero-crossings
    zc_real = (np.sign(r.real) * np.roll(np.sign(r.real),1)) < 0
    zc_imag = (np.sign(r.imag) * np.roll(np.sign(r.imag),1)) < 0
    zc = np.sum(zc_imag) + np.sum(zc_real)
    
    # Amplitude
    A = np.abs(r)           # amplitude
    mu_A = np.mean(A)       # mean amplitude
    A_N = A/mu_A            # normalized amplitude
    x_w = x[A_N<=A_t]       # indicies of weak samples
    x_nw = x[A_N>A_t]       # indicies of non-weak samples
    # indicies of non-weak samples of 2nd degree (previous sample also non-weak)
    x_nw2 = x_nw[x_nw-np.roll(x_nw,1)==1] 
    N_c = x_nw.size         # number of non-weak samples
    N_c2 = x_nw2.size        # number of non-weak samples of 2nd degree
    A_CN = A_N - 1          # centered normalized amplitude
    A_CN_nw = A_CN[x_nw]    # centered normalized amplitude of non-weak samples

    # PSD of Amplitude
    fs_A, F1_A_CN = dummyReceiver.calculate_spectrum(data=A_CN, n_fft=N_s)
    F1_A_CN = np.abs(F1_A_CN)
    fs_A, F2_A_CN = dummyReceiver.calculate_spectrum(data=A_CN**2, n_fft=N_s)
    F2_A_CN = np.abs(F2_A_CN)
    fs_A, F4_A_CN = dummyReceiver.calculate_spectrum(data=A_CN**4, n_fft=N_s)
    F4_A_CN = np.abs(F4_A_CN)
    
    # Phase
    # my interpretation
    phi = np.angle(r)
    phi_C = phi - np.mean(phi[x_nw]) 
    
    # the literature interpretation
    phi_uw = np.unwrap(phi)        
    phi_NL = phi_uw # no detrending because cfo is corrected
    phi_CNL = phi_NL - np.mean(phi_NL[x_nw])

    # Frequency
    f = (1/2*np.pi)*(phi_NL-np.roll(phi_NL, 1))/T_s
    mu_f = np.mean(f[x_nw2])
    f_CN = (f-mu_f)/(1/T_s)

    # Spectrogram
    fs, per = dummyReceiver.calculate_spectrum(data=r, n_fft=N_s)
    R = per[abs(fs) <= dummyReceiver.bw_signal/2]/N_s
    fs = fs[abs(fs) <= dummyReceiver.bw_signal/2]

    # autocorrelation
    Psi = (np.correlate(r, np.roll(r[:-500],50), mode='valid')/np.var(r)/N)
    #include noise in sigma!!!
    
    # Moments
    
    r20 = r10*r10
    r21 = r10*r11
    r22 = r11*r11
    r40 = r20*r20

    M20 = (1/N)*np.sum(r20)
    M21 = (1/N)*np.sum(r21)
    M22 = (1/N)*np.sum(r22)
    M40 = (1/N)*np.sum(r40)
    M41 = (1/N)*np.sum(r20*r21)
    M42 = (1/N)*np.sum(r20*r22)
    M60 = (1/N)*np.sum(r40*r20)
    M61 = (1/N)*np.sum(r40*r21)
    M62 = (1/N)*np.sum(r40*r22)
    M63 = (1/N)*np.sum(r20*r10*r22*r11)
    
    ########################################################################################
    # Features
    
    # Signal Features
    df.loc[i, "$N_c$ (1st)"] = N_c
    df.loc[i, "$N_c$ (2nd)"] = N_c2
    df.loc[i, "$ZC$"] = zc
    
    # Spectral Features
    df.loc[i, "$\gamma_{1,max}$"] = np.max(F1_A_CN)**2/N_s
    df.loc[i, "$\gamma_{2,max}$"] = np.max(F2_A_CN)**2/N_s
    df.loc[i, "$\gamma_{4,max}$"] = np.max(F4_A_CN)**2/N_s
    
    df.loc[i, "$\sigma_{aa}$"] = np.std(np.abs(A_CN))
    df.loc[i, "$\sigma_{a}$"] = np.std(A_CN[x_nw])
    df.loc[i, "$\sigma_{ap,CNL}$"] = np.std(np.abs(phi_CNL[x_nw]))
    df.loc[i, "$\sigma_{dp,CNL}$"] = np.std(phi_CNL[x_nw])
    df.loc[i, "$\sigma_{ap,C}$"] = np.std(np.abs(phi_C[x_nw]))
    df.loc[i, "$\sigma_{dp,C}$"] = np.std(phi_C[x_nw])
    df.loc[i, "$\sigma_{af}$"] = np.std(np.abs(f_CN[x_nw2]))

    df.loc[i, "$\mu^A_{42}$"] = scipy.stats.kurtosis(A_CN, fisher=False)
    df.loc[i, "$\mu^f_{42}$"] = scipy.stats.kurtosis(f_CN[x_nw2], fisher=False)
    
    df.loc[i, "$\Psi_{max}$"] = np.max(np.abs(Psi))
    df.loc[i, "$\sigma_{R}$"] = np.std(R)
    df.loc[i, "$\mu^R_{42}$"] = scipy.stats.kurtosis(R, fisher=False)
    df.loc[i, "$\\beta$"] = np.sum(r.real**2)/np.sum(r.imag**2)
    df.loc[i, "$C$"] = np.max(A)/np.sqrt(np.abs(M21))
    df.loc[i, "PAPR"] = np.abs(np.max(r21))/np.abs(M21)
    
    # Statistical Features
    df.loc[i, "$\hat{C}_{20}$"] = np.abs(M20)
    df.loc[i, "$\hat{C}_{21}$"] = np.abs(M21)
    
    df.loc[i, "$\\nu_{42}$"] = np.abs(M42/np.abs(M21))
    
    df.loc[i, "$|\widetilde{C}_{40}|$"] = np.abs((-3*M20**2 + M40)/np.abs(M21)**2)
    df.loc[i, "$|\widetilde{C}_{41}|$"] = np.abs((-3*M21*M20 + M41)/np.abs(M21)**2)
    df.loc[i, "$\widetilde{C}_{42}$"] = np.abs((-2*M21**2 - M22*M20 + M42)/np.abs(M21)**2)
    
    df.loc[i, "$|\widetilde{C}_{60}|$"] = np.abs((30*M20**3 - 15*M20*M40 + M60)/np.abs(M21)**3)
    df.loc[i, "$|\widetilde{C}_{61}|$"] = np.abs((30*M20**2*M21 - 14*M20*M41 - M40*M21 + M61)/np.abs(M21)**3)
    df.loc[i, "$|\widetilde{C}_{62}|$"] = np.abs((24*M21**2*M20 + 6*M22*M20**2 - 6*M20*M42 - 8*M21*M41 - M22*M40 + M62)/np.abs(M21)**3)
    df.loc[i, "$\widetilde{C}_{63}$"] = np.abs((12*M21**3 + 12*M22*M21*M20 - 9*M42*M21 + M63)/np.abs(M21)**3)
    
    
    ########################################################################################
    # display time domain characteristics of a single sample
    
    if(plot):
        x_nw = x_nw[x_nw<N_t] # non-weak timesamples
        x_nw2 = x_nw2[x_nw2<N_t] # non-weak timesamples (2nd degree)

        def ax_generator(num_plots):
            fig, axs = plt.subplots(num_plots, figsize=(14,int(4+num_plots*1.25)), sharex=False)
            for ax in axs:
                ax.grid(which='major', axis='x')
                ax.autoscale(axis='x', enable=False)
                ax.set_xlim([0, 1.15*N_t])
                ax.set_xticks(np.arange(0,N_t+1,int(N_t/10)))
                yield ax


        ax_gen = ax_generator(9)

        gca = next(ax_gen)
        gca.plot(r.real[:N_t], label="I")
        gca.plot(r.imag[:N_t], label="Q")
        gca.scatter(x[:N_t][zc_real[:N_t]], r.real[:N_t][zc_real[:N_t]], marker='.')
        gca.scatter(x[:N_t][zc_imag[:N_t]], r.imag[:N_t][zc_imag[:N_t]], marker='.')
        gca.set_title("r")
        gca.legend(loc=1)

        gca = next(ax_gen)
        gca.plot(A_N[:N_t], alpha=0.25)
        gca.scatter(x_nw, A_N[x_nw], color='r', marker='.', label="non-weak")
        gca.set_title("A_N")
        gca.legend(loc=1)

        gca = next(ax_gen)
        gca.plot(r10.real[:N_t], label="I", alpha=0.25)
        gca.plot(r10.imag[:N_t], label="Q", alpha=0.25)
        gca.scatter(x_nw, r.real[x_nw], color='r', marker='.', label="non-weak")
        gca.scatter(x_nw, r.imag[x_nw], color='r', marker='.', label="non-weak")
        gca.set_title("A_CN")
        gca.legend(loc=1)
        
        gca = next(ax_gen)
        gca.plot(phi_C[:N_t], alpha=0.25)
        gca.scatter(x_nw, phi_C[x_nw], color='r', marker='.', label="non-weak")
        gca.set_ylim([-3.5, 3.5])
        gca.set_title("phi_C (my phase interpretation)")
        gca.legend(loc=1)
        
        gca = next(ax_gen)
        gca.plot(phi_CNL[:N_t], alpha=0.25)
        gca.scatter(x_nw, phi_CNL[x_nw], color='r', marker='.', label="non-weak")
        gca.set_title("phi_CNL (textbook interpretation)")
        gca.legend(loc=1)

        gca = next(ax_gen)
        gca.plot(f_CN[:N_t], alpha=0.25)
        gca.scatter(x_nw, f_CN[x_nw], color='r', marker='.', label="non-weak\n1st degree")
        gca.scatter(x_nw2, f_CN[x_nw2], color='g', marker='.', label="non-weak\n2nd degree", alpha=0.5)
        gca.set_ylim([min(f_CN[x_nw2])*1.25, max(f_CN[x_nw2])*1.25])
        gca.set_title("f_CN")
        gca.legend(loc=1)

        # Spectrum
        gca = next(ax_gen)
        gca.plot(fs, R)
        gca.set_title("r_(f)")
        gca.legend(loc=1)
        gca.set_xlim([min(fs), max(fs)])
        gca.set_xticks(np.arange(min(fs),max(fs)+1,max(fs)/5))
        
        # Amplitude Spectrum
        gca = next(ax_gen)
        gca.plot(fs_A, F1_A_CN, label="F1_A_CN")
        gca.plot(fs_A, F2_A_CN, label="F2_A_CN")
        gca.plot(fs_A, F4_A_CN, label="F4_A_CN")
        
        gca.set_title("F_{[1,2,4],A_{CN}}")
        gca.legend(loc=1)
        gca.set_xlim([min(fs_A), abs(min(fs_A))])
        gca.set_xticks(np.arange(min(fs_A),abs(min(fs_A))+1,(abs(min(fs_A)))/5))
        gca.legend(loc=1)
        
        # Autocorrelation
        gca = next(ax_gen)
        gca.plot(np.abs(Psi))
        gca.set_title("\Psi_rr")
        gca.legend(loc=1)
        gca.set_xlim([0, len(Psi)])
        gca.set_xticks(np.arange(0, len(Psi)+1, int(len(Psi)/10))) 
        
        info("sys", "Showing the characteristics of a single sample")
        plt.tight_layout()
        plt.show()

 <hr>
 # Single file featurizer

In [ ]:
with ContextManager(streamhandler_level=logging.DEBUG, filehandler_path=home) as contextmanager:
    
    folder = "/storage/qnap/interference_free/ota/no_cfo/tx_usrp/rx_usrp/20db/npz/"
    file = "SC_BPSK_filtered.npz"
    data = np.load(folder+file)
    
    matrix = data['matrix']
    snrs = data['snrs']
    labels = data['labels']

    debug("OS", "opened %s, \nsize: %r" % (folder+file, matrix.shape))
    # dummyReceiver.visualize_samples(data = matrix[0,:])

    rows, cols = matrix.shape
    n_samps = 100
    debug("OS", "%d data samples with %d IQ samples each. Reading %d" % (rows, cols, n_samps))

    ########################################################################################
    # featurize samples    
    
    df = pd.DataFrame()
    _df = pd.DataFrame()
    
    df['SNR'] = snrs[:n_samps]
    
    for i, r in enumerate(matrix[:n_samps,:]):
        featurize_row(df, r)

        if(i % (n_samps/10) == 0):
            info("OS", "Progress: %d/%d" % (i,n_samps))    
    
    ########################################################################################
    # show time domain characteristic of one sample
    
    featurize_row(_df, matrix[0,:], plot=True, N_t=5000)
    
    ########################################################################################
    # display featurization
    

    for col in ["$N_c$ (1st)", "$N_c$ (2nd)"]:
        df[col] = df[col].astype(int)
    
    info("sys", "Showing the extracted features over all samples")
    
    pd.set_option('display.max_columns', None)  
    #pd.options.display.float_format = '{:,.2f}'.format
    display(df.iloc[[*np.arange(5), *np.arange(-5,0)],:])
    info("OS", "Dataframe size: %dx%d" % df.shape)

In [ ]:
feature = '$\hat{C}_{20}$'

i_min = df[df[feature] == np.min(df[feature].values)].index
i_max = df[df[feature] == np.max(df[feature].values)].index

print(i_min.values, i_max.values)
display(df.loc[i_min, :])
display(df.loc[i_max, :])

featurize_row(_df, matrix[i_min.values[0],:], plot=True, N_t=500)
featurize_row(_df, matrix[i_max.values[0],:], plot=True, N_t=500)

<hr>
# Batch featurizer 

In [ ]:
# batch featurizer

storage_dir = "/storage/qnap/"
root_folder = "interference_free/ota/no_cfo/tx_vsg/rx_sa/"
output_folder = "pkl/"
files = []

sample_size= -1 # numbers of samples to include
sample_lengths = [50000, 5000, 2500, 1250, 500] # (10k), 1k, 500, 250, 100 symbols

with ContextManager(streamhandler_level=logging.DEBUG, filehandler_path=home) as contextmanager:

    for root, directories, _ in os.walk(storage_dir + root_folder):
        for directory in directories:
            if(directory == "npz"):
                files += glob.glob(os.path.join(root, directory)+"/*filtered.npz")
            
    info("sys", "Found %d files" % len(files))
    
    for sample_length in sample_lengths:
        
        for iteration, file in enumerate(files):

            path, filename = re.search('(.*\/)([^\/]*)_filtered.npz$', file).groups()
            options = path[len(storage_dir):].split('/') + filename.split('_')

            # read file
            data = np.load(file)
            matrix = data['matrix']
            snrs = data['snrs']
            label = data['labels']
            sample_size = label.size if (sample_size == -1) else sample_size
            symbols = int(sample_length/5) # num of symbols
            debug("OS", "opened %s, size: %r" % (file[len(storage_dir):], matrix.shape))
            debug("OS", "options: %r" % (options))

            # build "X" Dataframe
            df = pd.DataFrame()
            df['SNR'] = snrs[:sample_size]
            for i, r in enumerate(matrix[:sample_size,:sample_length]):
                featurize_row(df, r)

            # build "Y" Dataframe     
            dfy = pd.DataFrame()

            dfy['symbols'] = [symbols]*sample_size

            for option in options:
                if (option == "cable" or option == "ota"):
                    dfy['connection'] = [option]*sample_size
                elif ("interference" in option):
                    dfy['interference'] = [option]*sample_size
                elif ("i_" in option):
                    dfy['interference'] = [option[2:]]*sample_size
                elif (option[-3:] == "sir"):
                    dfy['sir'] = [option[4:]]*sample_size            
                elif (option[-2:] == "db" or option[-2:] == "dB"):
                    dfy['snr_class'] = [option[:-2]+"db"]*sample_size
                elif (option[-3:] == "ppm" or option == "no_cfo"):
                    dfy['cfo'] = [option]*sample_size
                elif (option[:3] == "tx_"):
                    dfy['transmitter'] = option[3:]
                elif (option[:3] == "rx_"):
                    dfy['receiver'] = option[3:]
                elif (option[:5] == "intf_"):
                    dfy['interferer'] = option[5:]
                elif (option == "SC" or option == "OFDM"):
                    dfy['mod_type'] = [option]*sample_size
                elif (option[-3:] == "QAM" or option[-3:] == "PSK"):
                    dfy['mod'] = [option]*sample_size 
                elif (option in ["npz","","cfo", "interference"]):
                    pass # ignore
                else:
                    critical("sys", "unresolved option: %s" % option)
                    sys.exit(1)

            # write output file
            output_path = (path[:-len("npz/")]+output_folder).replace("qnap/", "qnap/untypical/lengths/")
            if not os.path.exists(output_path):
                os.makedirs(output_path)

            with open(output_path+filename+"_sym"+str(symbols)+".pkl", "wb") as pickle_file:
                pickle.dump({'X': df, 'y': dfy}, pickle_file)

            debug("OS", "output %r" % pickle_file.name)

            # iteration and reset buffers
            contextmanager.step(iteration+1, len(files))
            matrix = []
            snrs = []
            label = []


20.04. 14:05:55  INFO 	[sys]	Started Context Manager
20.04. 14:05:59  INFO 	[sys]	Found 40 files
20.04. 14:06:01  DEBG 	[OS]	opened interference_free/ota/no_cfo/tx_vsg/rx_sa/0db/npz/OFDM_16QAM_filtered.npz, size: (3200, 50000)
20.04. 14:06:01  DEBG 	[OS]	options: ['interference_free', 'ota', 'no_cfo', 'tx_vsg', 'rx_sa', '0db', 'npz', '', 'OFDM', '16QAM']
20.04. 14:08:06  DEBG 	[OS]	output '/storage/qnap/untypical/lengths/interference_free/ota/no_cfo/tx_vsg/rx_sa/0db/pkl/OFDM_16QAM_sym10000.pkl'
20.04. 14:08:06  INFO 	[sys]	Step 1/40	elapsed time: 00:02:10	time remaining: ~01:24:43)
20.04. 14:08:20  DEBG 	[OS]	opened interference_free/ota/no_cfo/tx_vsg/rx_sa/0db/npz/SC_BPSK_filtered.npz, size: (3200, 50000)
20.04. 14:08:20  DEBG 	[OS]	options: ['interference_free', 'ota', 'no_cfo', 'tx_vsg', 'rx_sa', '0db', 'npz', '', 'SC', 'BPSK']
20.04. 14:10:23  DEBG 	[OS]	output '/storage/qnap/untypical/lengths/interference_free/ota/no_cfo/tx_vsg/rx_sa/0db/pkl/SC_BPSK_sym10000.pkl'
20.04. 14:10:23  

20.04. 14:37:42  DEBG 	[OS]	output '/storage/qnap/untypical/lengths/interference_free/ota/no_cfo/tx_vsg/rx_sa/5db/pkl/SC_QPSK_sym10000.pkl'
20.04. 14:37:42  INFO 	[sys]	Step 14/40	elapsed time: 00:31:46	time remaining: ~00:59:00)
20.04. 14:37:56  DEBG 	[OS]	opened interference_free/ota/no_cfo/tx_vsg/rx_sa/5db/npz/OFDM_BPSK_filtered.npz, size: (3200, 50000)
20.04. 14:37:56  DEBG 	[OS]	options: ['interference_free', 'ota', 'no_cfo', 'tx_vsg', 'rx_sa', '5db', 'npz', '', 'OFDM', 'BPSK']
20.04. 14:40:00  DEBG 	[OS]	output '/storage/qnap/untypical/lengths/interference_free/ota/no_cfo/tx_vsg/rx_sa/5db/pkl/OFDM_BPSK_sym10000.pkl'
20.04. 14:40:00  INFO 	[sys]	Step 15/40	elapsed time: 00:34:04	time remaining: ~00:56:47)
20.04. 14:40:14  DEBG 	[OS]	opened interference_free/ota/no_cfo/tx_vsg/rx_sa/5db/npz/OFDM_QPSK_filtered.npz, size: (3200, 50000)
20.04. 14:40:14  DEBG 	[OS]	options: ['interference_free', 'ota', 'no_cfo', 'tx_vsg', 'rx_sa', '5db', 'npz', '', 'OFDM', 'QPSK']
20.04. 14:42:11  DEBG 

20.04. 15:07:40  DEBG 	[OS]	options: ['interference_free', 'ota', 'no_cfo', 'tx_vsg', 'rx_sa', '10db', 'npz', '', 'SC', '64QAM']
20.04. 15:09:44  DEBG 	[OS]	output '/storage/qnap/untypical/lengths/interference_free/ota/no_cfo/tx_vsg/rx_sa/10db/pkl/SC_64QAM_sym10000.pkl'
20.04. 15:09:44  INFO 	[sys]	Step 28/40	elapsed time: 01:03:48	time remaining: ~00:27:20)
20.04. 15:10:15  DEBG 	[OS]	opened interference_free/ota/no_cfo/tx_vsg/rx_sa/10db/npz/OFDM_64QAM_filtered.npz, size: (3200, 50000)
20.04. 15:10:15  DEBG 	[OS]	options: ['interference_free', 'ota', 'no_cfo', 'tx_vsg', 'rx_sa', '10db', 'npz', '', 'OFDM', '64QAM']
20.04. 15:12:24  DEBG 	[OS]	output '/storage/qnap/untypical/lengths/interference_free/ota/no_cfo/tx_vsg/rx_sa/10db/pkl/OFDM_64QAM_sym10000.pkl'
20.04. 15:12:24  INFO 	[sys]	Step 29/40	elapsed time: 01:06:28	time remaining: ~00:25:12)
20.04. 15:12:38  DEBG 	[OS]	opened interference_free/ota/no_cfo/tx_vsg/rx_sa/10db/npz/SC_QPSK_filtered.npz, size: (3200, 50000)
20.04. 15:12:38

20.04. 15:37:51  DEBG 	[OS]	opened interference_free/ota/no_cfo/tx_vsg/rx_sa/0db/npz/SC_BPSK_filtered.npz, size: (3200, 50000)
20.04. 15:37:51  DEBG 	[OS]	options: ['interference_free', 'ota', 'no_cfo', 'tx_vsg', 'rx_sa', '0db', 'npz', '', 'SC', 'BPSK']
20.04. 15:38:20  DEBG 	[OS]	output '/storage/qnap/untypical/lengths/interference_free/ota/no_cfo/tx_vsg/rx_sa/0db/pkl/SC_BPSK_sym1000.pkl'
20.04. 15:38:20  INFO 	[sys]	Step 2/40	elapsed time: 01:32:24	time remaining: ~05:15:36)
20.04. 15:38:33  DEBG 	[OS]	opened interference_free/ota/no_cfo/tx_vsg/rx_sa/0db/npz/SC_16QAM_filtered.npz, size: (3200, 50000)
20.04. 15:38:33  DEBG 	[OS]	options: ['interference_free', 'ota', 'no_cfo', 'tx_vsg', 'rx_sa', '0db', 'npz', '', 'SC', '16QAM']
20.04. 15:39:05  DEBG 	[OS]	output '/storage/qnap/untypical/lengths/interference_free/ota/no_cfo/tx_vsg/rx_sa/0db/pkl/SC_16QAM_sym1000.pkl'
20.04. 15:39:05  INFO 	[sys]	Step 3/40	elapsed time: 01:33:09	time remaining: ~19:08:51)
20.04. 15:39:19  DEBG 	[OS]	opene

20.04. 15:47:45  INFO 	[sys]	Step 15/40	elapsed time: 01:41:49	time remaining: ~02:49:42)
20.04. 15:48:00  DEBG 	[OS]	opened interference_free/ota/no_cfo/tx_vsg/rx_sa/5db/npz/OFDM_QPSK_filtered.npz, size: (3200, 50000)
20.04. 15:48:00  DEBG 	[OS]	options: ['interference_free', 'ota', 'no_cfo', 'tx_vsg', 'rx_sa', '5db', 'npz', '', 'OFDM', 'QPSK']
20.04. 15:48:32  DEBG 	[OS]	output '/storage/qnap/untypical/lengths/interference_free/ota/no_cfo/tx_vsg/rx_sa/5db/pkl/OFDM_QPSK_sym1000.pkl'
20.04. 15:48:32  INFO 	[sys]	Step 16/40	elapsed time: 01:42:36	time remaining: ~02:33:54)
20.04. 15:48:46  DEBG 	[OS]	opened interference_free/ota/no_cfo/tx_vsg/rx_sa/20db/npz/OFDM_16QAM_filtered.npz, size: (3200, 50000)
20.04. 15:48:46  DEBG 	[OS]	options: ['interference_free', 'ota', 'no_cfo', 'tx_vsg', 'rx_sa', '20db', 'npz', '', 'OFDM', '16QAM']
20.04. 15:49:15  DEBG 	[OS]	output '/storage/qnap/untypical/lengths/interference_free/ota/no_cfo/tx_vsg/rx_sa/20db/pkl/OFDM_16QAM_sym1000.pkl'
20.04. 15:49:15 

<hr>
# Refresh dfy's

In [ ]:
# refresh dfy's
storage_dir = "/storage/qnap/"
root_folder = "interference_free/"

folders = []
files = []

for root, directories, _ in os.walk(storage_dir + root_folder):
    for directory in directories:
        if(directory == "pkl"):
            folders.append(os.path.join(root, directory)+"/")
            files += glob.glob(os.path.join(root, directory)+"/*.pkl")

                
for iteration, file in enumerate(files):
            
    with open(file, "rb") as pickle_file:
        data=pickle.load(pickle_file)
    
    X = data['X'] # buffer
    sample_size = X.shape[0]    
    path, filename = re.search('(.*\/)([^\/]*).pkl$', file).groups()
    options = path[len(storage_dir):].split('/') + filename.split('_')

    # build "Y" Dataframe     
    dfy = pd.DataFrame()
    for option in options:
        if (option[:3] == "sym"):
            dfy['symbols'] = [option[3:]]*sample_size
        elif (option == "cable" or option == "ota"):
            dfy['connection'] = [option]*sample_size
        elif ("interference" in option):
            dfy['interference'] = [option]*sample_size
        elif ("i_" in option):
            dfy['interference'] = [option[2:]]*sample_size
        elif (option[-3:] == "sir"):
            dfy['sir'] = [option[4:]]*sample_size            
        elif (option[-2:] == "db" or option[-2:] == "dB"):
            dfy['snr_class'] = [option[:-2]+"db"]*sample_size
        elif (option[-3:] == "ppm" or option == "no_cfo"):
            dfy['cfo'] = [option]*sample_size
        elif (option[:3] == "tx_"):
            dfy['transmitter'] = option[3:]
        elif (option[:3] == "rx_"):
            dfy['receiver'] = option[3:]
        elif (option[:5] == "intf_"):
            dfy['interferer'] = option[5:]
        elif (option == "SC" or option == "OFDM"):
            dfy['mod_type'] = [option]*sample_size
        elif (option[-3:] == "QAM" or option[-3:] == "PSK"):
            dfy['mod'] = [option]*sample_size 
        elif (option in ["npz","","cfo", "interference"]):
            pass # ignore
        else:
            critical("sys", "unresolved option: %s" % option)
            sys.exit(1)

    # write output file
    with open(file, "wb") as pickle_file:
        pickle.dump({'X': X, 'y': dfy}, pickle_file)

    debug("OS", "output %r" % pickle_file.name)


<hr>
# Build database

In [ ]:
# build database

root_folders = [
    "/storage/qnap/interference_free/ota/no_cfo/tx_vsg/rx_sa/",
]

folders = []

for root_folder in root_folders:
    for root, directories, _ in os.walk(root_folder):
        for directory in directories:
            if(directory == "pkl"):
                folders.append(os.path.join(root, directory)+"/")

df = pd.DataFrame()
dfy = pd.DataFrame()

with ContextManager(streamhandler_level=logging.DEBUG, filehandler_path=home) as contextmanager:
    
    for folder in folders:
    
        files = glob.glob(folder+"*.pkl")

        for file in files:
            
            with open(file, "rb") as pickle_file:
                data=pickle.load(pickle_file)
                
            num_samples, num_features = data['X'].shape
            
            # HOTFIXES
            data['X'].rename(columns={'$\beta$': '$\\beta$', '$\nu_{42}$': '$\\nu_{42}$'}, inplace=True)
            data['y'].loc[:, 'snr_class'].replace({'0dB': '0db', '5dB': '5db', '10dB': '10db', '15dB': '15db', '20dB': '20db'}, inplace=True)
            if "cfo" not in data['y'].columns: data['y']['cfo'] = "no_cfo"
            if "sir" not in data['y'].columns: data['y']['sir'] = "none"
            if "interferer" not in data['y'].columns: data['y']['interferer'] = "none"
            if "symbols" not in data['y'].columns: data['y']['symbols'] = 10000
            if "label" in data['y'].columns: data['y'] = data['y'].drop("label", axis=1)
            
            df = df.append(data['X'])
            dfy = dfy.append(data['y'])           
    
    # fuse data in one big Dataframe
    tuples = list(zip(['X']*df.shape[1] + ['y']*dfy.shape[1], df.columns.append(dfy.columns)))
    df = pd.concat([df, dfy], axis=1)
    df.columns = pd.MultiIndex.from_tuples(tuples)
    df.reset_index(drop=True, inplace=True)
       
    with open("/storage/qnap/database_different_lengths.pkl", "wb") as pickle_file:
        pickle.dump({'df': df}, pickle_file)
        
    info("OS", "saved database to %s" % qnap)

#  visualize database

In [ ]:
print("Shape: (%d,%d)" % (df.shape))

dfy = df['y']
#dfy = drop_redundant_cols(dfy)
dfy = dfy.iloc[:,:-2].drop_duplicates()
dfy.reset_index(drop=True, inplace=True)
print("Showing all configuration permutations:")
display(dfy)

<hr>
<hr>
<hr>

# Junk

In [ ]:
from sklearn.cluster import KMeans

data = matrix[0,:]
data_rec =  np.copy(data)/np.amax(abs(data))

samples = 5000
results = []

for i in range(15):
    data_k = data_rec[0+i:samples+i:15]
    kmeans = KMeans(n_clusters=16, init='k-means++', n_init=10, max_iter=100, tol=0.0001).fit(np.r_['0,2', data_k.real, data_k.imag].T)
    #plt.scatter(data.real[2000+i:3000:15], data.imag[2000+i:3000:15])
    #plt.show()
    results.append(kmeans.inertia_)

i_fin = np.argmin(results)
data_fin = data_rec[0+i_fin:samples+i_fin:15]

fig, ax = plt.subplots()
ax.scatter(data_fin.real, data_fin.imag)
ax.set_aspect('equal', 'box')
plt.show()

In [ ]:
plt.plot([1,2,3])
plt.title("$\\beta$")
plt.show()